# io power - Samna only - v0.39.9

- Use PDM_CLK and PDM_DATA driven by FPGA to XA3:

1. Keep both PDM_DATA and PDM_CLK low：(0x2b,1),(0x2d,0),(0x2c,0)
2. Keep PDM_DATA high and PDM_CLK low: (0x2b,1),(0x2d,0),(0x2c,1)
3. Keep PDM_DATA low and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,2)
4. Keep PDM_DATA high and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,3)
5. Toggle PDM_DATA at some frequency while keeping PDM_CLK low:  (0x2b,1),(0x2d,1),(0x2c,0)
6. Toggle PDM_DATA at some frequency while keeping PDM_CLK high: (0x2b,1),(0x2d,1),(0x2c,1)
7. Keep PDM_DATA low while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,0)
8. Keep PDM_DATA high while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,2)
9. Toggle both PDM_CLK and PDM_DATA at some frequency: (0x2b,1),(0x2d,3)

### Initialization and connection to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')
import numpy as np

# Open the device and connect it to the power monitor.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")

power_monitor = xylo_node.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = xylo_node.get_stop_watch()
io = xylo_node.get_io_module()

xylo_model = xylo_node.get_model()
source = samna.graph.source_to(xylo_model.get_sink_node())

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo_model.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')

# XyloConfiguration
xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()

## With the network, clock frequency can not go low as 6.25
clock_frequencies = [12.5, 25, 50]
ts = 1e-3

power_measurements_625 = []
power_measurements_125 = []
power_measurements_25 = []
power_measurements_50 = []

Samna version 0.39.11.7+g04f769652


### Configure PDM_CLK in 1 MHz and MAIN_CLK

In [2]:
def configure_clocks():

    if (enable_main_clock):
        # Defines PDM clock rate
        # config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        # config.main_clock_frequency = 6250000
        # config.pdm_clock_frequency = 1000000
        # xylo_node.reset_board_soft(config)

        # configure main clock
        # io.write_config(0x0007, 7) # 6.25 MHz ->1100uW
        # configure main clock
        # io.write_config(0x0007, 15) # 3.125 MHz -> 20uW
        # io.write_config(0x0007, 13) # until here IO power is low MHz -> 20uW

        # configure main clock
        io.write_config(0x0007, 7) 

        time.sleep(1)
        io.write_config(0x0008, 1)  # main clock enable
        time.sleep(1)

    else:
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)

        # Disable main clock
        io.write_config(0x0008, 0)

def configure_clock_direction():
    if pdm_clock == "fpga":
        print("fpga -> xylo")
        xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0
    else:
        print("xylo -> fpga")
        xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

    xylo_node.get_model().apply_configuration(xylo_config)



### Map small network to the board

In [3]:
def configure_network():
    if map_network:
        input_count = 3
        hidden_count = input_count
        output_count = input_count
        xylo_config.input.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(input_count, hidden_count)
        xylo_config.hidden.weights = [[0, 0, 0], [0, 0, 0], [0, 0, 0]] # shape(hidden_count, hidden_count)
        xylo_config.hidden.neurons = [samna.xyloAudio3.configuration.HiddenNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * hidden_count
        xylo_config.readout.weights = [[127, 0, 0], [0, 127, 0], [0, 0, 127]] # shape(hidden_count, output_count)
        xylo_config.readout.neurons = [samna.xyloAudio3.configuration.OutputNeuron(threshold=127, v_mem_decay=1, i_syn_decay=1)] * output_count

    if digital_microphone:
        tr_wrap = 221
        xylo_config.operation_mode = samna.xyloAudio3.OperationMode.RealTime
        xylo_config.input_source = samna.xyloAudio3.InputSource.DigitalMicrophone
        xylo_config.time_resolution_wrap = tr_wrap
    
    xylo_node.get_model().apply_configuration(xylo_config)
    xylo_node.get_model().close_ram_access()

def configure_trigger_processing():

    if trigger_processing:
        source.write([samna.xyloAudio3.event.TriggerProcessing()])


### Use PDM_CLK and PDM_DATA driven by FPGA:
- No network mapped, and main clock is disabled
  


In [4]:
pdm_clock = "fpga"

map_network = True
trigger_processing = True
enable_main_clock = True

digital_microphone = True

In [5]:
def read_register(addr):
    # clear the buffer 
    sink.get_events()
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

ctrl1 = 0x0001
ctrl2 = 0x0002
ctrl3 = 0x0003
tr_wrap = 0x0004
hm_tr_wrap = 0x0005
clk_ctrl = 0x0006
clk_div = 0x0007
pwr_ctrl1 = 0x0008
pwr_ctrl2 = 0x0009
pwr_ctrl3 = 0x000A
pwr_ctrl4 = 0x000B
pad_ctrl = 0x000C
ie1 = 0x000E
ie2 = 0x000F
out_ctrl = 0x0011
monsel = 0x0166
mon_grp_sel = 0x0167
dbg_ctrl1 = 0x0168
dbg_stat1 = 0x0171
dfe_ctrl = 0x001B
ivgen = 0x0015
ivgen2 = 0x0016
ivgen3 = 0x0017
ivgen4 = 0x0018
ivgen5 = 0x0019
ivgen6 = 0x001A
adctest = 0x016E

# create access to xylo input/output
sink   = samna.graph.sink_from(xylo_model.get_source_node())

### 1 Keep both PDM_DATA and PDM_CLK low：(0x2b,1),(0x2d,0),(0x2c,0)


In [6]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,0)

# configure network to be mapped on chip, if selected
configure_network()

# xylo_node.get_model().close_ram_access()


# if enable_main_clock:
#     print(hex(read_register(27)))
    # print(hex(read_register(27)))

print("out_ctrl")
print(hex(read_register(out_ctrl)))
print(hex(read_register(12)))
print("pad_ctrl")
print(hex(read_register(pad_ctrl)))
print(hex(read_register(ctrl1)))


## IMPORTANT register to write to save power on digital core
wwv_ev = samna.xyloAudio3.event.WriteRegisterValue()
wwv_ev.address = out_ctrl
wwv_ev.data = 0x10
source.write([wwv_ev])

print("out_ctrl")
print(hex(read_register(out_ctrl)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
out_ctrl
0x70
0x0
pad_ctrl
0x0
0x1400000
out_ctrl
0x10
io:	1104 uW	AFE core:	13 uW	DFE+SNN core:	395 uW


### 2 Keep PDM_DATA high and PDM_CLK low: (0x2b,1),(0x2d,0),(0x2c,1)


In [7]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,1)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
io:	1107 uW	AFE core:	13 uW	DFE+SNN core:	396 uW


### 3 Keep PDM_DATA low and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,2)


In [8]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,2)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo
io:	1107 uW	AFE core:	13 uW	DFE+SNN core:	395 uW


### 4 Keep PDM_DATA high and PDM_CLK high: (0x2b,1),(0x2d,0),(0x2c,3)


In [9]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,0)
io.write_config(0x2c,3)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


fpga -> xylo


ZeroDivisionError: division by zero

### 5. Toggle PDM_DATA at some frequency while keeping PDM_CLK low:  (0x2b,1),(0x2d,1),(0x2c,0)


In [ ]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,1)
io.write_config(0x2c,0)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))


# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


### 6 Toggle PDM_DATA at some frequency while keeping PDM_CLK high: (0x2b,1),(0x2d,1),(0x2c,1)


In [ ]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,1)
io.write_config(0x2c,1)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


### 7 Keep PDM_DATA low while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,0)

In [ ]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,2)
io.write_config(0x2c,0)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


### 8 Keep PDM_DATA high while toggling PDM_CLK at some frequency: (0x2b,1),(0x2d,2),(0x2c,2)


In [ ]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,2)
io.write_config(0x2c,2)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


### 9 Toggle both PDM_CLK and PDM_DATA at some frequency: (0x2b,1),(0x2d,3)


In [ ]:
import time

# configure clocks
configure_clock_direction()
configure_clocks()

# configure PDM clock and PDM data
io.write_config(0x2b,1)
io.write_config(0x2d,3)

# configure network to be mapped on chip, if selected
configure_network()

# if enable_main_clock:
#     print(hex(read_register(27)))

# send or not a trigger processing based on configuration
configure_trigger_processing()


# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')
